In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
import numpy as np
import pandas as pd
import cv2
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
import tensorflow as tf

In [3]:
print(os.listdir("/gdrive/My Drive/DerinOgrenmeProje/lung_image_sets"))

['lung_squamous cell carcinoma', 'lung_adenocarcinoma', 'lung_benign tissue']


In [4]:
DATADIR = '/gdrive/My Drive/DerinOgrenmeProje/lung_image_sets'
CATEGORIES = ['lung_squamous cell carcinoma', 'lung_adenocarcinoma', 'lung_benign tissue']

In [5]:
IMG_SIZE = 100

In [6]:
all_data =[]

def create_all_data():
    for category in CATEGORIES:
        path=os.path.join(DATADIR, category)
        class_num=CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
                all_data.append([new_array,class_num])
            except Exception as e:
                pass
create_all_data()

In [7]:
print("Toplam Görüntü Sayısı: ", len(all_data))

Toplam Görüntü Sayısı:  15000


In [8]:
X = []
y = []

for categories, label in all_data:
  X.append(categories)
  y.append(label)

In [9]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state =42)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=42)

In [10]:
x_train = np.array(x_train)
x_test = np.array(x_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

x_val = np.array(x_val)
y_val = np.array(y_val)

In [11]:
train_yCl = tf.keras.utils.to_categorical(y_train, num_classes=3)
test_yCl = tf.keras.utils.to_categorical(y_test, num_classes=3)
valid_yCl = tf.keras.utils.to_categorical(y_val, num_classes=3)

In [12]:
resADD = tf.keras.applications.ResNet50V2(

    input_shape=(100, 100, 3),
    include_top = False

)
resADD.trainable = False

94668760/94668760 [==============================] - 0s 0us/step


In [13]:
res_model = Sequential()

res_model.add(resADD)
res_model.add(Dropout(0.4))
res_model.add(MaxPooling2D())
res_model.add(Flatten())
res_model.add(Dense(1024, activation='relu'))
res_model.add(BatchNormalization())
res_model.add(Dropout(0.2))
res_model.add(Dense(3, activation='softmax'))

res_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
res_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 4, 4, 2048)        23564800  
                                                                 
 dropout (Dropout)           (None, 4, 4, 2048)        0         
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 2, 2, 2048)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 1024)              8389632   
                                                                 
 batch_normalization (Batch  (None, 1024)              4096      
 Normalization)                                         

In [14]:
from tensorflow import keras

callback_list = [
    keras.callbacks.ModelCheckpoint(
        filepath='model.h5',
        monitor = 'val_accuracy', save_best_only=True, verbose=3
    ),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=3)
]

In [15]:
history = res_model.fit(x_train, train_yCl,
                    batch_size=128,
                    validation_data = (x_val, valid_yCl),
                    callbacks = callback_list,
                    epochs = 13
                    )

Epoch 1/13
80/80 [==============================] - ETA: 0s - loss: 0.4782 - accuracy: 0.8261
Epoch 1: val_accuracy improved from -inf to 0.60622, saving model to model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


80/80 [==============================] - 29s 242ms/step - loss: 0.4782 - accuracy: 0.8261 - val_loss: 2.2660 - val_accuracy: 0.6062
Epoch 2/13
79/80 [============================>.] - ETA: 0s - loss: 0.3429 - accuracy: 0.8575
Epoch 2: val_accuracy improved from 0.60622 to 0.82133, saving model to model.h5
80/80 [==============================] - 9s 107ms/step - loss: 0.3432 - accuracy: 0.8575 - val_loss: 0.3952 - val_accuracy: 0.8213
Epoch 3/13
79/80 [============================>.] - ETA: 0s - loss: 0.3235 - accuracy: 0.8686
Epoch 3: val_accuracy did not improve from 0.82133
80/80 [==============================] - 8s 101ms/step - loss: 0.3237 - accuracy: 0.8683 - val_loss: 0.5173 - val_accuracy: 0.7964
Epoch 4/13
79/80 [============================>.] - ETA: 0s - loss: 0.3076 - accuracy: 0.8740
Epoch 4: val_accuracy did not improve from 0.82133
80/80 [==============================] - 8s 98ms/step - loss: 0.3080 - accuracy: 0.8737 - val_loss: 1.1286 - val_accuracy: 0.6356
Epoch 5/13


In [16]:
res_model = keras.models.load_model('/content/model.h5')

In [17]:
score_train_res = res_model.evaluate(x_train, train_yCl)
print("Train Accuracy: ", score_train_res[1])

score_test_res = res_model.evaluate(x_test, test_yCl)
print("Test Accuracy: ", score_test_res[1])

score_val_res = res_model.evaluate(x_val, valid_yCl)
print("Validation Accuracy: ", score_val_res[1])

317/317 [==============================] - 12s 30ms/step - loss: 0.3867 - accuracy: 0.8261
Train Accuracy:  0.8260740637779236
118/118 [==============================] - 4s 38ms/step - loss: 0.4116 - accuracy: 0.8141
Test Accuracy:  0.81413334608078
36/36 [==============================] - 2s 53ms/step - loss: 0.3952 - accuracy: 0.8213
Validation Accuracy:  0.8213333487510681


In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

def rapor(model, x_test, test_yCl, categories):
    y_test_pred = model.predict(x_test)
    y_test_true = np.argmax(test_yCl, axis=1)
    y_test_pred = np.argmax(y_test_pred, axis=1)

    accuracy = accuracy_score(y_test_true, y_test_pred)
    precision = precision_score(y_test_true, y_test_pred, average='weighted')
    recall = recall_score(y_test_true, y_test_pred, average='weighted')
    f1 = f1_score(y_test_true, y_test_pred, average='weighted')

    conf_matrix = confusion_matrix(y_test_true, y_test_pred)
    class_report = classification_report(y_test_true, y_test_pred, target_names=categories)

    return accuracy, precision, recall, f1, conf_matrix, class_report


accuracy, precision, recall, f1, conf_matrix, class_report = rapor(res_model, x_test, test_yCl, CATEGORIES)
print(f"Test Accuracy: {accuracy}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall}")
print(f"Test F1 Skoru: {f1:.4f}\n")

print("\nKarmaşıklık Matrisi:")
print(conf_matrix)

print("\nSınıflandırma Raporu:")
print(class_report)

118/118 [==============================] - 3s 28ms/step
Test Accuracy: 0.8141333333333334
Test Precision: 0.8548
Test Recall: 0.8141333333333334
Test F1 Skoru: 0.8097


Karmaşıklık Matrisi:
[[ 731  549    4]
 [  25 1121   79]
 [   0   40 1201]]

Sınıflandırma Raporu:
                              precision    recall  f1-score   support

          lung_benign tissue       0.97      0.57      0.72      1284
lung_squamous cell carcinoma       0.66      0.92      0.76      1225
         lung_adenocarcinoma       0.94      0.97      0.95      1241

                    accuracy                           0.81      3750
                   macro avg       0.85      0.82      0.81      3750
                weighted avg       0.85      0.81      0.81      3750

